## IBDigital Open Question Analysis
This notebook presents an analysis of answers from IBDigital Questionnaire of the following questions:  
1. Reason of flare-ups? Would it help you to know in advance that you are going to have a flare-up? Explain why (not).  
2. What should a wearable not do? DO you have any ideas what a wearable should look like?  

We use topic models to anlayze frequency of answers.

### Data Preprocessing

In [1]:
# load data
import pandas as pd
data = pd.read_excel("data/DataSet_WP2_Excel.xlsx")
# display column BB to BD, and CF, DI
df = data.iloc[:, 53:56].join(data.iloc[:, 83:84]).join(data.iloc[:, 112:113])
df["Wish_predictability_flare"] = df["Wish_predictability_flare"].map({0: "Ja", 1: "Misschien", 2: "Nee"})
# check if there are line breakers in the text of all columns
for col in df.columns:
    if df[col].astype(str).str.contains('\n').any():
        print(f"Column {col} contains line breakers.")
        df[col] = df[col].str.replace('\n', ' ', regex=True)
# add ID column
df.insert(0, "ID", range(1, 1 + len(df)))
# save to csv
#df.to_csv("data/answers.csv", index=False)
# view dataframe
df.head()

Column Reason_flare contains line breakers.
Column Unwanted_features contains line breakers.
Column View_wearable contains line breakers.


,ID,Reason_flare,Wish_predictability_flare,Wish_predictability_because,Unwanted_features,View_wearable
0,1,Nee,Ja,minder klachten vooraf,geen extra belasting,smartwatch lijkt handig vrouwen hebben niet a...
1,2,Vroeger was dat bij stress,NaN,NaN,Ingewikkeld zijn.,Niet te groot en een mooi ontwerp
2,3,Het aan- of uitstaan van het familiaire coliti...,Ja,Dan kan ik mij AANMELDEN VOOR GERICHTE BEHANDE...,NaN,NaN
3,4,NaN,NaN,NaN,Bemoedigende woorden of tips geven die ik al 1...,Door het opvallend te maken is het meteen een ...
4,5,NaN,NaN,NaN,Aanwezig op de achtergrond ipv de voorgrond. T...,NaN


### Embed text
See embed_texts.py and embed.slurm

In [1]:
# load stopwords
with open("data/stopwords-nl.txt", "r") as f:
    stopwords = [line.strip() for line in f if line.strip()] # removed "geen" from stopwords

# remove "geen" from stopwords if present
if "geen" in stopwords:
    stopwords.remove("geen")

## 1. Unwanted Features

### 1.1 Load Data

In [10]:
# load embeddings and dfs
import numpy as np
import pandas as pd
embeddings_features = np.load("embeddings/emb_Unwanted_features.npy")
df_features = pd.read_csv("data/answers_for_embedding_Unwanted_features.csv")
texts_features = df_features["Unwanted_features"].astype(str).tolist()
print(f"Number of texts for Unwanted_features: {len(texts_features)}")
if embeddings_features.shape[0] != len(texts_features):
    raise ValueError("Number of embeddings does not match number of texts.")
# set colwidth to 200
pd.set_option('display.max_colwidth', 200)
df_features.head()

Number of texts for Unwanted_features: 860


,ID,Unwanted_features
0,1,geen extra belasting
1,2,Ingewikkeld zijn.
2,4,Bemoedigende woorden of tips geven die ik al 1000 keer gehoord of gelezen heb
3,5,Aanwezig op de achtergrond ipv de voorgrond. Te veel meldingen. Mij te veel confronteren met mijn ziekte.
4,6,periodiek gedurende de dag meldingen blijven geven zonder dat er iets aan de hand is. Veel meldingen met allerlei adviezen geven. Daar zou ik echt stress van krijgen waardoor het averechts werkt.


### 1.2 Create Semantic Groups
Strict grouping (cosine threshold = 0.93) of near-duplicate sentences

In [65]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

TEXT_COL = "Unwanted_features"

df = df_features.copy()
df[TEXT_COL] = df[TEXT_COL].astype(str).fillna("").str.strip()
df = df[df[TEXT_COL].ne("") & df[TEXT_COL].str.lower().ne("nan")].copy()

E = np.asarray(embeddings_features, dtype=np.float32)
assert len(df) == E.shape[0], "df and embeddings_features must align row-wise"

TEXT_COL = "Unwanted_features"

df = df_features.copy()
df[TEXT_COL] = df[TEXT_COL].astype(str).fillna("").str.strip()
df = df[df[TEXT_COL].ne("") & df[TEXT_COL].str.lower().ne("nan")].copy()

E = np.asarray(embeddings_features, dtype=np.float32)
assert len(df) == E.shape[0], "df and embeddings_features must align row-wise"

def semantic_groups_by_threshold(embeddings, threshold=0.93, batch_size=1000):
    """
    Returns: group_id per row, and groups dict {gid: [idxs...]}
    Uses a similarity graph + connected components.
    """
    n = embeddings.shape[0]
    embs = normalize(embeddings)  # cosine-ready

    # Build adjacency list (batched)
    adj = [[] for _ in range(n)]
    for start in range(0, n, batch_size):
        end = min(start + batch_size, n)
        sims = cosine_similarity(embs[start:end], embs)  # (b, n)

        for i in range(end - start):
            idx = start + i
            neighbors = np.where(sims[i] >= threshold)[0]
            # remove self
            neighbors = neighbors[neighbors != idx]
            adj[idx].extend(neighbors.tolist())

    # Connected components (DFS)
    group_id = -np.ones(n, dtype=int)
    gid = 0
    for i in range(n):
        if group_id[i] != -1:
            continue
        stack = [i]
        group_id[i] = gid
        while stack:
            u = stack.pop()
            for v in adj[u]:
                if group_id[v] == -1:
                    group_id[v] = gid
                    stack.append(v)
        gid += 1

    groups = {}
    for i, g in enumerate(group_id):
        groups.setdefault(int(g), []).append(i)
    return group_id, groups

df["SemGroup"], sem_groups = semantic_groups_by_threshold(E, threshold=0.93)

def representative_by_centroid(idxs, embeddings):
    embs = embeddings[idxs]
    centroid = embs.mean(axis=0, keepdims=True)
    sims = cosine_similarity(embs, centroid).reshape(-1)
    return idxs[int(sims.argmax())], float(sims.max())

sem_rows = []
for g, idxs in sem_groups.items():
    rep_idx, rep_sim = representative_by_centroid(idxs, E)
    sem_rows.append({
        "SemGroup": g,
        "Count": len(idxs),
        "Representative": df.iloc[rep_idx][TEXT_COL],
        "Rep_centroid_sim": rep_sim,
        "Variants": df.iloc[idxs][TEXT_COL].head(8).tolist(),  # show a few
    })

semantic_top_answers = (
    pd.DataFrame(sem_rows)
      .sort_values("Count", ascending=False)
      .reset_index(drop=True)
)

print(len(semantic_top_answers), "semantic groups found.")
semantic_top_answers.head(20)

674 semantic groups found.


,SemGroup,Count,Representative,Rep_centroid_sim,Variants
0,16,82,Te veel meldingen,0.994626,"[Te veel meldingen, Te veel meldingen, Te veel meldingen, Te veel meldingen, Teveel meldingen, Te veel meldingen, Te veel meldingen, Te veel meldingen]"
1,1,36,Ingewikkeld zijn,0.999349,"[Ingewikkeld zijn., Ingewikkeld zijn, Ingewikkeld zijn, Ingewikkeld zijn, Ingewikkeld zijn., Ingewikkeld zijn, Ingewikkeld zijn, Ingewikkeld zijn]"
2,93,16,stress geven,0.999463,"[stress geven, Stress geven., Stress geven., Stress geven, Stress geven, Stress geven, stress geven, Stress geven]"
3,54,13,Te ingewikkeld zijn,0.999747,"[Te ingewikkeld zijn, Te ingewikkeld zijn, te ingewikkeld zijn, Te ingewikkeld zijn, Te ingewikkeld zijn, Te ingewikkeld zijn, te ingewikkeld zijn, Te ingewikkeld zijn.]"
4,21,12,Geen idee,1.000000,"[Geen idee, geen idee, Geen idee, Geen idee, geen idee, Geen idee, Geen idee, geen idee]"
5,149,8,"Te veel meldingen, ingewikkeld",0.986079,"[Te ingewikkeld en teveel meldingen, Te veel meldingen, te ingewikkeld in gebruik, Veel meldingen, ingewikkeld zijn, Te veel meldingen en te ingewikkeld, Teveel meldingen, ingewikkeld zijn, Te vee..."
6,34,5,Te veel tijd kosten,0.986365,"[Veel tijd kosten, Te veel tijd kosten, Te veel tijd kosten, Teveel tijd kosten, Te veel tijd kosten]"
7,176,4,"Stress geven, te veel meldingen geven, ingewikkeld zijn",0.984584,"[Stress geven, te veel meldingen en ingewikkeld zijn., te veel meldingen, ingewikkeld zijn, stress geven., Stress geven, te veel meldingen geven, ingewikkeld zijn, Stress geven, te veel meldingen,..."
8,285,3,Niet ingewikkeld zijn,0.992787,"[Niet ingewikkeld zijn., Niet ingewikkeld zijn, Niet ingewikkeld zijn]"
9,67,3,niet te ingewikkeld,0.994991,"[Niet te ingewikkeld, Niet te ingewikkeld zijn, niet te ingewikkeld]"


### 1.3 Clustring the Semantic Groups using KMeans

In [66]:
num_topics = 15 # Number of clusters

topic_model = KMeans(n_clusters=num_topics, random_state=0, n_init="auto")
df["Topic"] = topic_model.fit_predict(E)

centers = topic_model.cluster_centers_
E_norm = normalize(E)
C_norm = normalize(centers)

# Similarity to own topic centroid
own_centroid_sim = (E_norm * C_norm[df["Topic"].values]).sum(axis=1)
df["Topic_centroid_sim"] = own_centroid_sim

# Get topic examples 
def topic_examples(df_topic, top_k=3):
    # 1) most central
    central = df_topic.sort_values("Topic_centroid_sim", ascending=False).head(1)

    # 2) diverse: farthest from centroid but still in topic (helps show variation)
    diverse = df_topic.sort_values("Topic_centroid_sim", ascending=True).head(top_k-1)

    ex = pd.concat([central, diverse]).drop_duplicates(subset=[TEXT_COL]).head(top_k)
    return ex[TEXT_COL].tolist()

topic_examples_map = (
    df.groupby("Topic", group_keys=False)
      .apply(lambda x: topic_examples(x, top_k=3), include_groups=False)
)


In [67]:
from scipy.sparse import csr_matrix

topic_docs = df.groupby("Topic")[TEXT_COL].apply(lambda s: " ".join(s)).tolist()

cv = CountVectorizer(stop_words=stopwords, ngram_range=(1,2), min_df=2)
X = cv.fit_transform(topic_docs)  # (n_topics, vocab)
vocab = np.array(cv.get_feature_names_out())

# c-TF-IDF
# tf: counts per topic
tf = X

# df: in how many topics term appears
df_term = np.asarray((X > 0).sum(axis=0)).ravel()
N = X.shape[0]
idf = np.log((N + 1) / (df_term + 1)) + 1.0  # smoothed IDF

# tf-idf-like weighting for class-based docs
ctfidf = tf.multiply(idf)  # sparse

def top_ctfidf_words(topic_id, top_n=10):
    row = ctfidf.getrow(topic_id)
    if row.nnz == 0:
        return []
    top_idx = np.argsort(row.data)[::-1][:top_n]
    return vocab[row.indices[top_idx]].tolist()

topic_keywords = {t: ", ".join(top_ctfidf_words(t, 12)) for t in range(num_topics)}

topic_counts = df["Topic"].value_counts().reindex(range(num_topics), fill_value=0)

topic_summary = pd.DataFrame({
    "Topic": range(num_topics),
    "Count": topic_counts.values,
    "Keywords_cTFIDF": [topic_keywords.get(t, "") for t in range(num_topics)],
}).merge(topic_examples_df, left_on="Topic", right_index=True, how="left")

topic_summary = topic_summary.sort_values("Count", ascending=False).reset_index(drop=True)
print("Topic summary:")
topic_summary.head(30)


Topic summary:


,Topic,Count,Keywords_cTFIDF,Example_1_Central,Example_2_Diverse,Example_3_Diverse
0,4,126,"meldingen, meldingen meldingen, teveel meldingen, meldingen geven, teveel, geven meldingen, meldingen teveel, geven, meldingen ingewikkeld, ingewikkeld, ingewikkeld meldingen, ingewikkeld teveel","Te veel meldingen, ingewikkeld",Teveel toetsen moeten aanklikken,Veel storingen geven
1,13,118,"meldingen, meldingen geven, geven, teveel meldingen, teveel, melding, meldingen teveel, onnodige meldingen, onnodige, constant, dag, gaat",stress geven,Onrust veroorzaken en spanningen oproepen.,Me zorgen maken dat ik iets fout doe wat juist meer stress geeft
2,10,79,"aanwezig, beperken, opvallen, groot, leidend, last, lomp, gebruik, lelijk groot, zie, zitten, bezig",Zorgen dat je teveel met je ziekte bezig bent,"Je wilt niet de hele dag, constant, ermee bezig zijn. Ik ga meer dan 8 keer per dag naar de wc. Heb vrijwel continu buikpijn. Ben continu te moe en moet overdag bijslagen. Toch wil ik ook iets and...",Dat je er last van krijgt en moet niet te veel aanwezig zijn. Want vind wel dat je niet te gefuseerd daarop moet zijn. Dan heb je ook geen leven.
3,11,75,"meldingen, dragen, mogelijk, horloge, gebruik, makkelijk, geven, gaat, geen, draag, app, apparaat",Stress geven( te veel informatie),Nog meer onzekerheden brengen,"Stress geven. Het is belangrijk dat een wearable duidelijk uitleg geeft over bijvoorbeeld een stress score. Anders kan het juist stress opleveren, en dan heeft het een averechts effect. Positieve ..."
4,7,61,"ziekte, bezig, stress, ziekte bezig, gaat, dag, teveel, meldingen, geven, continue, continu, mee bezig",Geen stress geven,stress veroorzaken. Onhandig/groot in gebruik.,Stress geven en het moet niet te ingewikkeld zijn
5,1,61,"meldingen, stress, geven, stress geven, geven meldingen, teveel, meldingen ingewikkeld, teveel meldingen, meldingen stress, ingewikkeld, geven stress, meldingen geven",Te ingewikkeld zijn,"Te ingewikkeld, of veel foutieve metingen","Lelijk, groot, ingewikkeld, engels talig"
6,0,60,"stress, geven, meldingen, geen, stress geven, geen stress, teveel, geven geen, opvlamming, waarschuwen, ingewikkeld, extra",geen stress en zeker niet teveel meldingen,Goede vraag! Reclames bevatten. Dit zou ik heel irritant vinden en dan stop ik er gelijk mee. Verder kan ik niets bedenken.,Zorgen dat ik niet overprikkeld raakt door veel dezelfde vragen .
7,14,55,"stress, stress geven, geven, geven stress, ingewikkeld stress, geven geen, geven ingewikkeld, ingewikkeld, geven teveel, veroorzaken, stress veroorzaken, teveel",Niet te ingewikkeld en niet teveel meldingen geven want dat kan weer stress veroorzaken waardoor een opvlamming kan worden getriggerd.,Te snel Alarm geeft. Samenvatting eind vd dag is voldoende,Invloed hebben op je lichaam door straling of iets
8,12,45,"ingewikkeld, ingewikkeld ingewikkeld, moeilijk, gebruik, gebruik ingewikkeld, geven ingewikkeld, moeilijk gebruik, geven, groot, ingewikkeld gebruik, stress, stress geven",Overbodige informatie geven.,Data verkopen aan Google oid.,Verkeerde interpretabele informatie geven
9,6,43,"informatie, informatie geven, data, geven, gegevens, delen, derden, valse, info, wearable, geven informatie, arts",Te veel meldingen,"Te veel meldingen geven, trillingen.",Teveel meldt


### 1.4 Overview of Topic Distribution
Share = proportion of all responses that belong to a topic  
Cumulative share = how much of the total dataset is covered by the top N topics (in order)

In [72]:
topic_summary["Share"] = topic_summary["Count"] / topic_summary["Count"].sum()
topic_summary["CumShare"] = topic_summary["Share"].cumsum()
topic_summary["Keywords_cTFIDF"] = topic_summary["Keywords_cTFIDF"].str.slice(0, 60) + "..."
topic_summary[["Topic", "Count", "Share", "CumShare", "Keywords_cTFIDF"]].head(15)

,Topic,Count,Share,CumShare,Keywords_cTFIDF
0,4,126,0.146512,0.146512,"meldingen, meldingen meldingen, teveel meldingen, meldingen ..."
1,13,118,0.137209,0.283721,"meldingen, meldingen geven, geven, teveel meldingen, teveel,..."
2,10,79,0.091860,0.375581,"aanwezig, beperken, opvallen, groot, leidend, last, lomp, ge..."
3,11,75,0.087209,0.462791,"meldingen, dragen, mogelijk, horloge, gebruik, makkelijk, ge..."
4,7,61,0.070930,0.533721,"ziekte, bezig, stress, ziekte bezig, gaat, dag, teveel, meld..."
5,1,61,0.070930,0.604651,"meldingen, stress, geven, stress geven, geven meldingen, tev..."
6,0,60,0.069767,0.674419,"stress, geven, meldingen, geen, stress geven, geen stress, t..."
7,14,55,0.063953,0.738372,"stress, stress geven, geven, geven stress, ingewikkeld stres..."
8,12,45,0.052326,0.790698,"ingewikkeld, ingewikkeld ingewikkeld, moeilijk, gebruik, geb..."
9,6,43,0.050000,0.840698,"informatie, informatie geven, data, geven, gegevens, delen, ..."
